In [ ]:

data_type="sprial"
penalty="MMD"
#penalty="WASS"
center=10
latent_dim=1
intermediate_dim=128
input_dim=2

# data_type="torus"
# center=15
# lanten_dim=2
# input_dim=3


In [ ]:
!pip install POT
import ot
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import math, os
from numpy import array as matrix, arange
from numpy import zeros, ones,empty
import tensorflow_datasets
import random
import math
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from matplotlib import pyplot as plt
import math, os
from numpy import array as matrix, arange
from numpy import zeros, ones,empty
import tensorflow_datasets
import random
import math
import sklearn
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from tensorflow.keras import datasets, layers, models,constraints
from scipy.stats import truncnorm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 664 kB 4.3 MB/s 


In [ ]:
#generate training data
import matplotlib.pyplot as plt
if data_type=="sprial":
  r= 1
  phi = 3 * np.pi * np.random.rand(1000)
  x = r * np.cos(phi+2)*phi/(np.pi)
  y = r * np.sin(phi+2)*phi*2/(np.pi)
  D1=np.array([x,y]).T
  x_train=D1
if data_type=="torus":
  R= 3
  r=1
  data=[]
  n=3000
  o=0
  iend=0
  phi = 2* np.pi * np.random.rand(n)
  theta=2* np.pi * np.random.rand(n)
  x=(R+r*np.cos(theta))*np.cos(phi)
  y=(R+r*np.cos(theta))*np.sin(phi)
  z=r*np.sin(theta)
  D1=np.array([x,y,z]).T
  x_train=D1
  

In [ ]:
##find partition of unity

from sklearn.cluster import MiniBatchKMeans
kmeans = sklearn.cluster.MiniBatchKMeans(n_clusters=center,batch_size=1000)
kmeans = kmeans.fit(x_train)
A=kmeans.cluster_centers_


In [ ]:
nn=x_train.shape[0]
gg=zeros((center,nn))
for i in range(nn):
  for j in range(center):
    gg[j,i]=np.dot(np.transpose(x_train[i,:]-A[j,:]),(x_train[i,:]-A[j,:]))
label=zeros((nn,1))
for i in range(nn):
  label[i,0]=np.argmin(gg[:,i])
radius=zeros((center,1))
for i in range(center):
  radius[i,0]=np.max(gg[i,np.where(label[:,0]==i)])**0.5+1
rhopar=10
def rho(xindex,r,cindex,gg):
  z=gg[cindex,xindex]**0.5
  if r>=z:
    return (r**2-z**2)**rhopar
  else:
    return 0

def trho(index,xindex,gg):
 z=0
 for j in range(center):
   z=z+rho(xindex,radius[j,0],j,gg)
 return rho(xindex,radius[index,0],index,gg)/z
prob=zeros((center,1))
for j in range(center):
  for i in  range(nn):
      prob[j,0]=prob[j,0]+trho(j,i,gg)/nn
nprob=empty((center,1))
for j in range(center):
   nprob[j,0]=round(prob[j,0]*100)
nprob[np.argmax(nprob)]=nprob[np.argmax(nprob)]+100-sum(nprob)
nnprob=empty((center,1))
for j in range(center):
   nnprob[j,0]=round(prob[j,0]*10000)

In [ ]:
def pourho0(index,x):
 zz1=tf.expand_dims(x-A[index,:],1)
 zz2=tf.expand_dims(x-A[index,:],2)
 zz=tf.squeeze(tf.matmul(zz1,zz2)**0.5,axis=2)
 return  tf.cast(radius[j,0]>=zz,tf.float32)*(radius[j,0]**2-zz**2)**rhopar
def pourho(index,x):
 z=0
 for j in range(center):
   z=z+pourho0(j,x)
 return pourho0(index,x)/z

In [ ]:
def compute_kernel(x, y):
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    dim = tf.shape(x)[1]
     
    tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1]))
    tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))
def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return tf.reduce_mean(x_kernel)+ tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel) 

In [ ]:
if data_type=="sprial":
  def compute_wass(x,y):
    val=0
    x1=tf.sort(x,axis=0)
    y1=tf.sort(y,axis=0)
    
    val=val+tf.reduce_mean(tf.abs(x1-y1))
    

    return val
    
if data_type=="torus":
  def compute_wass(x,y):
    M=(tf.reduce_sum(tf.abs(tf.expand_dims(x,1) - tf.expand_dims(y,0)), 2))
    u1 = ot.utils.unif(x.shape[0])
    u2 = ot.utils.unif(y.shape[0])
    M1=M.numpy()
    w=ot.emd(u1,u2,M1)
    val=tf.reduce_sum(w*M)

    return val

In [ ]:

encoder_inputs = keras.Input(shape=(input_dim))
encoder_outputs = layers.Dense(units=intermediate_dim, activation='relu')(encoder_inputs)
encoder_cov = keras.Model(encoder_inputs, encoder_outputs, name="encoder_cov")
encoder_cov.summary()
encoder_linear_inputs=keras.Input(shape=(intermediate_dim,))
encoder_linear_outputs = layers.Dense(latent_dim*center)(encoder_linear_inputs)
encoder_linear = keras.Model(encoder_linear_inputs, encoder_linear_outputs, name="encoder_linear")
encoder_linear.summary()
 

Model: "encoder_cov"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 128)               512       
                                                                 
Total params: 512
Trainable params: 512
Non-trainable params: 0
_________________________________________________________________
Model: "encoder_linear"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_1 (Dense)             (None, 15)                1935      
                                                                 
Total params: 1,935
Trainable par

In [ ]:
  
latent_inputs = keras.Input(shape=(latent_dim,))
decoder_linear_outputs = layers.Dense(intermediate_dim*center, activation='relu')(latent_inputs)
decoder_linear = keras.Model(latent_inputs, decoder_linear_outputs, name="decoder_linear")
decoder_linear.summary()
decoder_inputs= keras.Input(shape=(intermediate_dim,))

decoder_outputs =layers.Dense(input_dim)(decoder_inputs)
 
decoder_cov = keras.Model(decoder_inputs, decoder_outputs, name="decoder_cov")
decoder_cov.summary()

Model: "decoder_linear"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 dense_2 (Dense)             (None, 1920)              3840      
                                                                 
Total params: 3,840
Trainable params: 3,840
Non-trainable params: 0
_________________________________________________________________
Model: "decoder_cov"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128)]             0         
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 387
Trainable p

In [ ]:
####prior $\nu_0$######

def samplelatent(n,latent_dim,j=0):
    r=3
    x=tf.constant(truncnorm.rvs(-r,r,size=(n, latent_dim)),dtype='float32')
    return x
  

In [ ]:
 
class VAE(keras.Model):
    def __init__(self, encoder_cov,encoder_linear, decoder_cov,decoder_linear, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder_cov = encoder_cov
        self.decoder_cov = decoder_cov
        self.encoder_linear = encoder_linear
        self.decoder_linear = decoder_linear
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.penalty_loss_tracker = keras.metrics.Mean(name="penalty_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.penalty_loss_tracker,
        ]

    def train_step(self, data):
          
        data1=tf.reshape(data,(batch_size,input_dim))
        rhovalue=[]
        for i in range(center):
          a=tf.sqrt(tf.reduce_sum(tf.square(data1-tf.transpose(tf.repeat(tf.expand_dims(A[i,],-1),data.shape[0],axis=1))),axis=1))
          rhovalue.append(tf.math.multiply(tf.cast(tf.where(radius[i,0]>=a, 1, 0),'float32'),(radius[i,0]**2-a**2)**rhopar))
        b=tf.reduce_sum(tf.stack(rhovalue),axis=0)
        for i in range(center):
          rhovalue[i]=rhovalue[i]/b
        
        x=[]
        batch=[]
        for centerite in range(center):
            kk=tf.where(tf.where(tf.random.uniform([data.shape[0]])<rhovalue[centerite],0,1)==0) 
            x.append(tf.gather_nd(data, kk))
            batch.append(kk.shape[0])
      
        x=tf.concat(x,axis=0)
        with tf.GradientTape() as tape:
            meanlist=encoder_linear(encoder_cov(x))
            meanlist=tf.split(meanlist,batch,axis=0)
            for i in range(center):
                    meanlist[i]=meanlist[i][:,(i*latent_dim):((i+1)*latent_dim)]
            meanlistconcat=tf.concat(meanlist,axis=0)
            predsini=decoder_linear(meanlistconcat)
            predsini=tf.split(predsini,batch,axis=0)
            for i in range(center):
                    predsini[i]=predsini[i][:,(i*intermediate_dim):((i+1)*intermediate_dim)]
            predsini=tf.concat(predsini,axis=0)
            predslist=decoder_cov(predsini)
            x=tf.split(x,batch,axis=0)
            predslist=tf.split(predslist,batch,axis=0)
            reconstruction_loss=0
            for i in range(center):
                      reconstruction_loss=reconstruction_loss+center*keras.metrics.mean_squared_error(K.flatten(x[i]), K.flatten(predslist[i]))*prob[i]
            penalty_loss=0
            for i in range(center):
               if penalty=="MMD":
                    penalty_loss= penalty_loss+10*compute_mmd(samplelatent(meanlist[i].shape[0], latent_dim,i), meanlist[i])
               if penalty=="WASS":
                    penalty_loss= penalty_loss+compute_wass(samplelatent(meanlist[i].shape[0], latent_dim,i), meanlist[i])
            total_loss=K.mean(reconstruction_loss + penalty_loss)
              
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.penalty_loss_tracker.update_state(penalty_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "penalty_loss": self.penalty_loss_tracker.result(),
        }


In [ ]:
 
 
x_train=x_train.astype('float32')
label=label
A=A.astype('float32')
 
batch_size=200
vae = VAE(encoder_cov, encoder_linear,decoder_cov,decoder_linear)
vae.compile(optimizer=keras.optimizers.Adam(0.001),run_eagerly=True)
vae.fit(x_train,epochs=1000, batch_size=200)


In [ ]:
#####update prior###

import copy
decoder_cov1=copy.deepcopy(decoder_cov)
decoder_linear1=copy.deepcopy(decoder_linear)
####data-driven prior######

def samplelatent(n,latent_dim,j):
    r=3
 
    iend=0
    x=[]
    o=0
    for i in range(100):
     if iend==0:
        data=tf.constant(truncnorm.rvs(-r,r,size=(3*n, latent_dim)),dtype='float32')
        aaa=(tf.random.uniform(shape=[data.shape[0],latent_dim])<=pourho(j,decoder_cov1(decoder_linear1(data)[:,(j*intermediate_dim):((j+1)*intermediate_dim)])))
        ind=(tf.cast(tf.where(aaa==True)[:,0],tf.int32))
        data=tf.gather(data,ind)
        o=o+ind.shape[0]
        x.append(data)
        if o>=n:
          iend=1
    x=tf.concat(x,axis=0)[0:n,:]
    
    return x
###train the model with the the data-driven prior####
vae = VAE(encoder_cov, encoder_linear,decoder_cov,decoder_linear)
vae.compile(optimizer=keras.optimizers.Adam(0.001),run_eagerly=True)
vae.fit(x_train,epochs=1000, batch_size=200)

In [ ]:
### Compute W_1 distance between generated samples and test samples###
r= 1
phi = 3 * np.pi * np.random.rand(30000)



x = r * np.cos(phi+2)*phi/(np.pi)
y = r * np.sin(phi+2)*phi*2/(np.pi)
D1=np.array([x,y]).T
x_test=D1

z_sample=[]
for i in range(center):
  z_sample.append(samplelatent(int(np.round(nnprob[i]*3)), latent_dim,i))
a=decoder_cov(decoder_linear(z_sample[0])[:,(0*intermediate_dim):((0+1)*intermediate_dim)])
for i in range(1,center):
  a=tf.concat([decoder_cov(decoder_linear(z_sample[i])[:,(i*intermediate_dim):((i+1)*intermediate_dim)]),a],axis=0)
a=a.numpy()
u1 = ot.utils.unif(a.shape[0])
u2 = ot.utils.unif(x_test.shape[0])
M=ot.dist(x_test,a,metric='minkowski',p=1)
w = ot.emd2(u2, u1, M)
print(w)